In [1]:
import json
import w3lib.html
import random
import os
from datetime import datetime
import requests
import string
import calendar
import html
import locale 
from requests_html import HTMLSession
from requests_html import HTML
import validators
import pandas as pd
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
URL_SEED = 'https://www.meganoticias.cl/nacional/?page=1'
XPATH_HREF = "//figcaption/a/@href"

In [3]:
class Crawler():
    
    USER_AGENT_LIST = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]

    #Ruta XPATH hacia los enlaces que queremos recopilar
    xpath_href=XPATH_HREF

    #variable que permite listar todos los enlaces recopilado
    url_list = []
    
    def _init_(self, URL_SEED, XPATH_HREF):        
        pass

    
    def get(self,url):
        session = HTMLSession()
        header = dict()
        header['user-agent'] = random.choice(self.USER_AGENT_LIST)
        response = session.get(url,headers=header)        
        return response
    
    def parse(self, response):
        self.url_list = []
        urls = response.html.xpath(self.xpath_href)
        for url in urls:
            self.url_list.append(url)


In [4]:
urls=[]

for page in tqdm(range(3)): # revisemos sólo 2 páginas
    crawler = Crawler()
    response = crawler.get('https://www.meganoticias.cl/nacional/?page='+str(page))
    crawler.parse(response)

    
    for url in crawler.url_list:
        urls.append(url)

    crawler.url_list = urls
    
crawler.url_list

  0%|          | 0/3 [00:00<?, ?it/s]

['https://www.meganoticias.cl/nacional/372032-cadem-encuesta-gabriel-boric-aprobacion-desaprobacion-gobierno-27-03-2022.html',
 'https://www.meganoticias.cl/nacional/372057-temblor-epicentro-hoy-copiapo-region-de-atacama-27-03-2022.html',
 'https://www.meganoticias.cl/nacional/372052-encuesta-cadem-gabriel-boric-barbara-figueroa-evaluacion-embajadora-argentina-27-03-2022.html',
 'https://www.meganoticias.cl/nacional/372051-barbara-figueroa-embajadora-en-argentina-presidente-gabriel-boric-cancilleria-27-03-2022.html',
 'https://www.meganoticias.cl/nacional/372046-temblor-el-tabon-navidad-hoy-27-03-2022.html',
 'https://www.meganoticias.cl/nacional/372036-crisis-migratoria-colchane-bebe-mujer-mueren-cruzar-chile-27-03-2022.html',
 'https://www.meganoticias.cl/nacional/372042-daniel-zamudio-10-anos-de-su-muerte-mama-pedido-presidente-boric-ley-antidiscriminacion-27-03-2022.html',
 'https://www.meganoticias.cl/nacional/372039-primos-tralcal-beneficio-salida-dominica-crimen-luchsinger-macka

In [5]:
len(urls)

54

In [6]:
urls[0]

'https://www.meganoticias.cl/nacional/372032-cadem-encuesta-gabriel-boric-aprobacion-desaprobacion-gobierno-27-03-2022.html'

In [8]:
URL_NEWS = urls[0]
XPATH_TITLE = "//h1"
XPATH_CONTENT ="//div[@class='contenido-nota']/p"

In [9]:
class Scraper:
    
    USER_AGENT_LIST = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]

    query_extract_title = XPATH_TITLE
    query_extract_text =  XPATH_CONTENT

    def _init_(self):
        pass
   
    def get(self,url):
        headers = {'user-agent':random.choice(self.USER_AGENT_LIST) }
        session = HTMLSession()
        response = session.get(url,headers=headers) 
        return response
    
    def clean_text(self, html_text):
        text_without_tags = w3lib.html.remove_tags(html_text)
        text_without_escape_chars = w3lib.html.replace_escape_chars(text_without_tags)
        text_without_escape_chars = html.unescape(text_without_escape_chars)
        text_without_whitespace = text_without_escape_chars.strip()
        return text_without_whitespace

    
    #### main function ####
    def parse(self, response):
        
        #URL
        self.news_url = response.url
        
        #TITLE
        self.news_title = response.html.xpath(self.query_extract_title)[0].text
        self.news_title = self.clean_text(self.news_title)
       
        
        #CONTENT
        self.news_text = ""
        
        html_paragraphs = response.html.xpath(self.query_extract_text)
        
        for paragraph in html_paragraphs:
            paragraph_content = w3lib.html.remove_tags(paragraph.text)    
            text_cleaned = self.clean_text(paragraph_content)
            self.news_text = self.news_text+text_cleaned


In [10]:
scraper = Scraper()
response =scraper.get(URL_NEWS)

In [11]:
scraper.parse(response)

In [12]:
scraper.news_title

'Cadem: Aprobación de Boric se mantiene estable aunque desaprobación aumenta 10 puntos en una semana'

In [13]:
scraper.news_text

'Meganoticias adelanta en exclusiva los resultados de la encuesta Plaza Pública Cadem de este domingo 27 de marzo, que revela que el Presidente Gabriel Boric mantiene un 50% de aprobación, pero su desaprobación aumenta 10 puntos, pasando del 20 al 30%.Ante la pregunta: "Independiente de su posición política, ¿Usted aprueba o desaprueba la forma como Gabriel Boric está conduciendo su gobierno?", un 50% de los encuestados aseguró que "Aprueba", al igual que la semana pasada.En contrapartida, un 30% desaprueba la forma en que el Mandatario está conduciendo su Gobierno, el que comenzó el pasado 11 de marzo. Esto significa un aumento de 10 puntos porcentuales respecto a la medición anterior, cuando esta respuesta llegaba a un 20%.Te puede interesarCadem: 57% de encuestados le "parece mal" que Boric nombrara a Bárbara Figueroa como embajadora en ArgentinaMinistro Jackson y quinto retiro: "No queremos que los trabajadores paguen las crisis con sus propios recursos"Mascarillas: ¿Qué dice la mi

In [14]:
#Verificamos que el scraper devuelve algo
assert(len(scraper.news_title)>0)
assert(len(scraper.news_text)>0)

In [15]:
df = pd.DataFrame(data={'url': [URL_NEWS],'title':[scraper.news_title], 'content':[scraper.news_text]})
df

,url,title,content
0,https://www.meganoticias.cl/nacional/372032-ca...,Cadem: Aprobación de Boric se mantiene estable...,Meganoticias adelanta en exclusiva los resulta...


In [17]:
XPATH_TITLE = "//h1"
XPATH_CONTENT ="//div[@class='contenido-nota']/p"

for url in tqdm(urls):
    URL_NEWS = url
    scraper = Scraper()
    response =scraper.get(URL_NEWS)
    scraper.parse(response)
    df1 = pd.DataFrame(data={'url': url, 'title':scraper.news_title, 'content':scraper.news_text},index=['url', 'title', 'content'])
    df = df.append(df1, ignore_index=True)
df = df.drop_duplicates()


  0%|          | 0/54 [00:00<?, ?it/s]

In [18]:
df

,url,title,content
0,https://www.meganoticias.cl/nacional/372032-ca...,Cadem: Aprobación de Boric se mantiene estable...,Meganoticias adelanta en exclusiva los resulta...
7,https://www.meganoticias.cl/nacional/372057-te...,Temblor afecta a la zona norte del país,"La noche de este 27 de marzo, a las 21:37 hora..."
10,https://www.meganoticias.cl/nacional/372052-en...,"Cadem: 57% de encuestados le ""parece mal"" que ...",Meganoticias adelanta en exclusiva los resulta...
13,https://www.meganoticias.cl/nacional/372051-ba...,Presidente Gabriel Boric confirma a Bárbara Fi...,"Este domingo, Gabriel Boric confirmó a la ex p..."
16,https://www.meganoticias.cl/nacional/372046-te...,Temblor se registra en la zona central del país,"La tarde de este domingo 27 de marzo, a las 15..."
19,https://www.meganoticias.cl/nacional/372036-cr...,Crisis migratoria en Colchane: Bebé de 6 meses...,"La crisis migratoria en Colchane, región de Ta..."
22,https://www.meganoticias.cl/nacional/372042-da...,A 10 años del crimen de Daniel Zamudio: Madre ...,En la presente jornada se cumplieron 10 años d...
25,https://www.meganoticias.cl/nacional/372039-pr...,Abogado de primos Tralcal y salida dominical: ...,"El abogado de los primos Tralcal, condenados a..."
28,https://www.meganoticias.cl/nacional/372038-se...,Por decimotercera ocasión: Tribunal estadounid...,El expresidente de la Asociación Nacional de F...
31,https://www.meganoticias.cl/nacional/372033-co...,"Conmemoración de ""caso degollados"": Ministra s...",Este domingo se realizó un acto conmemorativo ...
